### 1. 사용자 입력 받기
사용자로부터 탑승 클래스, 여행 구간, 서비스 선호도, 리뷰 분석 여부를 입력받습니다.
입력된 데이터는 이후 필터링 및 추천 점수 계산에 사용됩니다.


In [1]:
def get_user_input():
    print("사용자 정보를 입력하세요:")
    
    # 필수 입력 항목
    seat_type = input("탑승 클래스 (Economy Class, Premium Economy, Business Class, First Class): ").strip()
    route = input("여행 구간 (예: Seoul to New York): ").strip()
    analyze_reviews = input("리뷰 분석 여부 (Yes/No): ").strip().lower() == "yes"
    
    # 선호도 입력
    print("서비스 선호도를 입력하세요 (1~5 순위):")
    preferences = {
        "Seat Comfort": int(input("좌석 편안함 (1~5): ").strip()),
        "Cabin Staff Service": int(input("객실 승무원 서비스 (1~5): ").strip()),
        "Food & Beverages": int(input("기내 음식 및 음료 (1~5): ").strip()),
        "Inflight Entertainment": int(input("기내 엔터테인먼트 (1~5): ").strip()),
        "Ground Service": int(input("지상 직원 서비스 (1~5): ").strip()),
        "Wifi & Connectivity": int(input("기내 와이파이 (1~5): ").strip()),
        "Value For Money": int(input("가격 대비 가치 (1~5): ").strip()),
    }
    
    return {
        "seat_type": seat_type,
        "route": route,
        "analyze_reviews": analyze_reviews,
        "preferences": preferences,
    }

user_input = get_user_input()


사용자 정보를 입력하세요:
서비스 선호도를 입력하세요 (1~5 순위):


### 2. 데이터 로드 및 전처리
JSON 데이터 파일을 로드한 후, 필요한 열만 유지하고 결측치를 처리합니다.
Route 필드를 정규화하여 다양한 형식을 통일합니다.


### 3. Route 정규화 및 매칭
Route 값을 정규화하고 입력값과 비교하여 출발지와 도착지가 일치하는지 확인합니다.
경유지(`via`)는 무시합니다.


In [2]:
def normalize_route(route):
    """
    Route 값을 정규화합니다.
    - null 값은 빈 문자열로 대체.
    - 숫자 값은 빈 문자열로 처리.
    - via 이후 경유지를 제거.
    - to, 공백 등을 제거하여 단순화.
    """
    if pd.isnull(route) or isinstance(route, (int, float)):
        return ""
    if "via" in route.lower():
        route = route.split("via")[0].strip()
    return route.lower().replace("to", "").replace(" ", "")


def extract_route_parts(route):
    """
    Route에서 출발지와 도착지를 추출합니다.
    """
    if not route:
        return "", ""
    parts = route.split("to")
    start = parts[0].strip() if len(parts) > 0 else ""
    end = parts[1].strip() if len(parts) > 1 else ""
    return start, end

def match_route(input_route, data_route):
    """
    입력값과 데이터셋의 Route를 비교합니다.
    - 출발지와 도착지가 동일한 경우 True 반환.
    """
    input_start, input_end = extract_route_parts(normalize_route(input_route))
    data_start, data_end = extract_route_parts(normalize_route(data_route))
    return (input_start == data_start and input_end == data_end) or \
           (input_start == data_end and input_end == data_start)


### 4. 데이터 필터링
입력된 탑승 클래스와 여행 구간을 기준으로 데이터셋을 필터링합니다.


In [3]:
def filter_data(df, user_input):
    """
    사용자 입력값에 따라 데이터를 필터링합니다.
    """
    seat_type = user_input["seat_type"].strip().lower()
    input_route = user_input["route"].strip()
    
    # 데이터 정리
    df["Seat Type"] = df["Seat Type"].str.strip().str.lower()
    
    # 탑승 클래스 필터링
    filtered_df = df[df["Seat Type"] == seat_type]
    
    # 여행 구간 필터링
    if input_route:
        filtered_df = filtered_df[filtered_df["Route"].apply(lambda r: match_route(input_route, r))]
    
    return filtered_df

filtered_data = filter_data(df, user_input)

if filtered_data.empty:
    print("조건에 맞는 데이터가 없습니다. 입력 조건을 확인하세요.")
else:
    print("필터링된 데이터:")
    print(filtered_data.head(50))


NameError: name 'df' is not defined

### 5. 리뷰 감성 분석
BERT 모델을 사용하여 리뷰 텍스트를 분석하고 감성 점수를 추가합니다.


In [ ]:
import sys
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def analyze_review_sentiment_pytorch_progress(df, analyze_reviews):
    """
    PyTorch 기반 BERT 모델로 리뷰 감성 분석을 수행합니다.
    - 리뷰가 없거나 비어 있는 경우 0점 반환.
    - GPU를 사용하여 처리 속도를 향상시킵니다.
    - 진행률은 같은 줄에서 실시간으로 업데이트됩니다.
    """
    if not analyze_reviews:
        print("리뷰 분석이 비활성화되었습니다.")
        return df
    
    # GPU 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # 모델 로드
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    model.to(device)

    def predict_sentiment(review):
        if pd.isnull(review) or not review.strip():
            return 0
        inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        sentiment_score = torch.argmax(probs, dim=-1).item() + 1
        return sentiment_score

    # 감성 점수 계산 및 진행률 출력
    total_reviews = len(df)
    sentiment_scores = []
    for index, review in enumerate(df["Review Content"]):
        sentiment_scores.append(predict_sentiment(review))
        percent_complete = ((index + 1) / total_reviews) * 100
        sys.stdout.write(f"\rProgress: {percent_complete:.2f}% ({index + 1}/{total_reviews})")
        sys.stdout.flush()
    
    print()  # 진행 완료 후 줄바꿈
    df["Sentiment Score"] = sentiment_scores
    return df

# 리뷰 감성 분석 수행
filtered_data = analyze_review_sentiment_pytorch_progress(filtered_data, user_input["analyze_reviews"])


Using device: cuda
Progress: 100.00% (3/3)


### 6. 추천 점수 계산
필터링된 데이터와 감성 점수를 기반으로 가중 평균 점수를 계산합니다.


In [ ]:
def calculate_recommendation_score(df, user_input):
    preferences = user_input["preferences"]
    total_weight = sum(preferences.values())
    
    # 가중치 계산
    weights = {k: v / total_weight for k, v in preferences.items()}
    
    df["Recommendation Score"] = (
        df["Seat Comfort"] * weights["Seat Comfort"] +
        df["Cabin Staff Service"] * weights["Cabin Staff Service"] +
        df["Food & Beverages"] * weights["Food & Beverages"] +
        df["Inflight Entertainment"] * weights["Inflight Entertainment"] +
        df["Ground Service"] * weights["Ground Service"] +
        df["Wifi & Connectivity"] * weights["Wifi & Connectivity"] +
        df["Value For Money"] * weights["Value For Money"] +
        df.get("Sentiment Score", 0)
    )
    
    return df

scored_data = calculate_recommendation_score(filtered_data, user_input)


###  7. 항공사 추천
추천 점수를 기반으로 상위 5개의 항공사를 추천합니다.


In [ ]:
def recommend_airlines(df):
    airline_scores = df.groupby("Airline Name")["Recommendation Score"].mean().sort_values(ascending=False)
    return airline_scores.head(5)

recommendations = recommend_airlines(scored_data)
print("추천 항공사:")
print(recommendations)

추천 항공사:
Airline Name
Air Busan    8.023810
Eastarjet    6.380952
Name: Recommendation Score, dtype: float64
